In [ ]:
# import all packages
import scipy,corner,emcee,nestle,warnings
import pandas
import scipy.optimize
import scipy.stats
import matplotlib.pyplot as plt
import numpy as np
warnings.simplefilter('ignore')

In [ ]:
# read in some data we created for this example (.dat is a generic filename, it's just a text file)
example_data_1D = pandas.read_csv('1D_intro_examples.dat',sep=' ',header=0)#this file is separated by spaces and its first line contains the names of the columns (header) 
print(example_data_1D.head())

In [ ]:
#Let's plot the data, with error bars, that we read from file (See Day 2)
plt.errorbar(example_data_1D['x'], #x,y,and error are the column names
             example_data_1D['y'], 
             yerr=example_data_1D['error'],#yerr denotes an error in the y-direction for plotting
             fmt='.') #fmt is "format", saying that I want data marked by "points"
plt.xlabel('x') #set the x-axis label 
plt.ylabel('y') #set the y-axis label
plt.show()

In [ ]:
#The data were generated with a simple quadratic equation:
#ax^2+bx+c. The true model values are:
a_true=2
b_true=1.3
c_true=6
def my_model(x,a,b,c): #We define the model described above
    return(a*x**2+b*x+c)

**Data Fitting: Chi-Square Minimization**
===============================

In [ ]:
#Let's try and fit the data with a chi-square minimization
def chisq_likelihood(theta, args):
    #This function accepts an argument "theta", which is 
    #a list of model parameters a, b and c. It then calculates
    #a chi-square statistic that it returns, which compares
    #the observations, errors, and model provided in args.
    
    x, y, yerr,mod = args #args is a list, so this is the same as x=args[0],y=args[1],yerr=args[2]. x,y, and yerr are numpy arrays, mod is a function.
    a,b,c = theta #theta is also a list, so it follows the same as args above
    model_observations = mod(x,a,b,c) #mod (a model) is the 4th element of args, and it accepts x values, and the three model parameters a,b,c. Now model_observations contains the model values at every point in x (and is a numpy array)
    inv_sigma2 = 1./yerr**2 #The chi-square statistic contains an inverse-square error, which we calculate here
    chisquare = np.sum((y-model_observations)**2*inv_sigma2 )#calculate the chi-square statistic. 
    return chisquare

#Use scipy.optimize.minimize to minimize the chi-square, which
#is the same as maximizing the likelihood.
result = scipy.optimize.minimize(chisq_likelihood, #the first argument is the function to minimize, which must accept a list of parameters as its first argument, and any other necessary agruments as its second.
                                 x0=[1,1,1], #x0 is a first guess for each of your parameters (depends on your model/data)
                                 bounds=[(-100,100),(-100,100),(-100,100)], #optional bounds for each of your parameters
                                 args=[example_data_1D['x'],#the args passed to chisq_likelihood above
                                       example_data_1D['y'],
                                       example_data_1D['error'],
                                       my_model])#my_model is defined in a previous cell

print(result)#scipy.optimize.minimize returns a dictionary which I've called result. It has other information, the "x" element has the best fit values
a_ml,b_ml,c_ml = result["x"] #get our best fit values from result

#set up plotting the model over the data
plt.errorbar(example_data_1D['x'],
             example_data_1D['y'],
             yerr=example_data_1D['error'],
             fmt='.',
             label='Data')

plt.plot(example_data_1D['x'],
         my_model(example_data_1D['x'],a_ml,b_ml,c_ml),
         'g--',#make the line green and dashed
         label='$\chi^2$ Fit')

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
print('Chi-square fit:',a_ml, b_ml,c_ml)
print('True:',a_true,b_true,c_true)
plt.show()

In [ ]:
#Now let's look at a drawback of the chi-square minimization.
#This fitting technique depends on finding a minimum in the chi-square.    
#What if there is some sort of local minimum in the chi-square?

#my_bad_model was used to create new data, read in below.
#This model has one of the parameters squared, 
#so either -b or b will give the same result
def my_bad_model(x,a,b,c):
    return(a*x**2+b**2*x+c)#b is now squared 

#These are the true values used to make the data
a_true_bad=2
b_true_bad=-3.3 #note b_true_bad is negative here
c_true_bad=6

#Read in some data generated with this model. 
my_bad_data_1D=pandas.read_csv("1D_intro_examples_bad.dat",header=0,sep=' ')

#Plot the data
plt.errorbar(my_bad_data_1D['x'],my_bad_data_1D['y'],yerr=my_bad_data_1D['error'],fmt='.')
plt.xlabel('x')
plt.ylabel('y')
plt.show()


In [ ]:
#Try and fit the data with this model
result_bad = scipy.optimize.minimize(chisq_likelihood, 
                                     [1,1,1],
                                     bounds=[(-100,100),(-100,100),(-100,100)], 
                                     args=[my_bad_data_1D['x'],
                                           my_bad_data_1D['y'],
                                           my_bad_data_1D['error'],
                                           my_bad_model])

a_ml, b_ml,c_ml = result_bad["x"] #get our best fit values

#Plot the bad data
plt.errorbar(my_bad_data_1D['x'],
             my_bad_data_1D['y'],
             yerr=my_bad_data_1D['error'],
             fmt='.',
             label='Data')
#Plot our fitting result
plt.plot(my_bad_data_1D['x'],
         my_bad_model(my_bad_data_1D['x'],a_ml,b_ml,c_ml),
         'g--',
         label='$\chi^2$ Fit')#note we can use latex directly in labels for matplotlib
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
print('Chi-square fit:',a_ml, b_ml,c_ml)
print('True:',a_true_bad,b_true_bad,c_true_bad)
plt.show()
#However, in this (very contrived) case, the true value of b is -3.3

# Data Fitting: Markov Chain Monte Carlo (MCMC)

In [ ]:
# We can combine a likelihood function with a prior function.

# Create a log-prior function. 
def lnprior(theta):#accepts the model parameters (theta)
    a,b,c = theta #set a,b,c (see above)
    
    #give the following bounds: a=(0,5),b=(-5,5),c=(0,10)
    if 0 < a < 5. and -5 < b < 5 and 0 < c < 10: #we are assuming a "uniform prior" on all parameters, which is the same as just giving each parameter bounds.
        return 0.0 #if you try parameters inside the bounds, return a probability of 1 (log(1)=0)
    return -np.inf #if you try parameters outside the bounds, return 0 (log(0)=-inf)

# This is a log-likelihood function, which is commonly used.
def lnprob(theta, x, y, yerr,mod): #accepts theta (the model parameters), and the same x,y,yerr, and mod from above
    lp = lnprior(theta) #get the probability from the prior function
    if not np.isfinite(lp): 
        return -np.inf #return a probability of negative infinity if the prior is negative infinity

    #the chisq_likelihood function returns a chi-square, 
    #which you want to be as small as possible. We are 
    #maximizing the likelihood here, so we take
    #the negative of the chi-square function.
    return lp - chisq_likelihood(theta, [x, y, yerr,mod]) #the total likelihood is the product of the prior and the likelihood (or the sum of the log-prior and log-likelihood)


#Set up the MCMC to sample the full parameter space
ndim, nwalkers = 3, 100 #number of parameters to fit (3); number of individual "walkers" that randomly sample the space. Choose any number, the higher the slower.
starting_positions = [result["x"] + 0.1*np.random.randn(ndim) for i in range(nwalkers)] #Start the walkers in a random (small) gaussian near the result from the chi-square fit

#Let's just look at where walkers are starting in the a-b parameter space
#(Doesn't need to be included in your own code)
#Notice we get a random sampling near our previous result to start. 
plt.scatter([x[0] for x in starting_positions],#get the "a" parameter locations
            [x[1] for x in starting_positions])#get the "b" parameter locations
plt.scatter(result['x'][0],result['x'][1],color='r')#the chi-square result
plt.xlabel('a')
plt.ylabel('b')
plt.show()


In [ ]:
#set up the MCMC sampler
sampler = emcee.EnsembleSampler(nwalkers, 
                                ndim, 
                                lnprob, #the likelihood function to maximize 
                                args=(example_data_1D['x'], #the arguments passed to the likelihood functions (other than the model parameters)
                                      example_data_1D['y'], 
                                      example_data_1D['error'],
                                      my_model))

output = sampler.run_mcmc(starting_positions, 500) #run the MCMC with the starting positions we defined and 500 sampling points per walker


In [ ]:
#sampler.chain contains all of the samples from the MCMC.
print(sampler.chain.shape)
#It currently holds the samples separately for each walker.
#We don't care about what each walker does, so let's flatten it:
samples = sampler.chain.reshape((-1, ndim)) #The -1 here means we don't care how many rows it takes, give us the same number of columns as we have parameters
print(samples.shape)
#So we tried 50000 total model realizations for 3 parameters

In [ ]:
#So what did we actually get from this? Let's use another
#python package to see the output of the MCMC sampling
fig = corner.corner(samples, #samples is defined above
                    labels=["$a$", "$b$","$c$"],#parameter labels
                    quantiles=(0.16,0.50,0.84),#plot some quantiles for reference
                    truths=[a_true, b_true,c_true])#show the true values for comparison
plt.show()

In [ ]:
#But how close did we get to the true values?
#We can take the 50th percentile of the distributions you see above
#As our result. Incidentally we could calculate the uncertainty as 
#well, perhaps as the differences between the 84th and 50th for
#the upper uncertainty, and 50th and 16th for the lower uncertainty
a_mcmc, b_mcmc,c_mcmc = np.percentile(samples, 50,axis=0)#axis=0 means we want to calculate percentiles along columns, not rows
print('Chi:','a=',a_ml,'b=', b_ml,'c=',c_ml)
print('MCMC:','a=',a_mcmc,'b=',b_mcmc,'c=',c_mcmc)
print('True:','a=',a_true,'b=', b_true,'c=',c_true)
#Definitely closer

In [ ]:
#Now let's see how the MCMC deals with the "bad" data/model above
#The set up is exactly the same as before.

ndim, nwalkers = 3, 100
#I've increased the range around the chi-square result
#That I want my walkers to start at because I know 
#We're far from the right answer. You could always make
#The walkers start over a wide range to start with,
#look at the posterior, and hone in on the correct area
#of parameter space. 
pos = [result_bad["x"] - 2*np.abs(np.random.randn(ndim)) for i in range(nwalkers)]

sampler_bad = emcee.EnsembleSampler(nwalkers, 
                                    ndim, 
                                    lnprob, 
                                    args=(my_bad_data_1D['x'], 
                                          my_bad_data_1D['y'], 
                                          my_bad_data_1D['error'],
                                          my_bad_model))
sampler_bad.run_mcmc(pos, 500)
samples_bad = sampler_bad.chain.reshape((-1, ndim))

In [ ]:
#Let's look at the result
a_mcmc_bad, b_mcmc_bad,c_mcmc_bad = np.percentile(samples_bad, 50,axis=0)
print('MCMC:','a=',a_mcmc_bad,'b=',b_mcmc_bad,'c=',c_mcmc_bad)
print('True:','a=',a_true_bad,'b=', b_true_bad,'c=',c_true_bad)
fig = corner.corner(samples_bad, labels=["$a$", "$b$","$c$"],
                      truths=[a_true_bad, b_true_bad,c_true_bad])
plt.show()

# Bayesian Nested Sampling

In [ ]:
#An alternative to MCMC is Bayesian Nested Sampling. It is
#extremely similar but randomly samples the full parameter
#space instead of sending out individual walkers. MCMC
#is often better for a many-dimensional model.

# Define a likelihood function. Let's use a chi-square again.
# The package doing the fitting needs a likelihood function
# that only accepts the parameters, so we can create this 
# small wrapper function that calls our original chisq_likelihood
# function with the necessary arguments.
def loglike(theta):
    args=(example_data_1D['x'],
          example_data_1D['y'],
          example_data_1D['error'],
          my_model)
    chisq = chisq_likelihood(theta,args)
    return -1.*chisq #again we need to take the negative of the chi-square because we need a function to maximize

#This can be a bit difficult to understand. We are using the same bounds
#on our parameters as before here [a=(0,5),b=(-5,5),c=(0,10)].
#However, nested sampling always reduces the problem to the 
#range (0,1) for every parameter. Then we always need to provide
#a function prior_transform, which applies our prior to the model
#and simultaneously tranforms from this unit space, to our actual
#parameter space. This essentially means rescaling the parameter 
#from the range (0,1) to the range (-5,5).
#
#For example, we want a uniform prior on b, meaning bounds from -5 to 5.
#The algorithm will only try values for b from 0 to 1. Suppose it tries
#b=0.5. That's halfway between 0 and 1, which means it is halfway between
#our bounds (-5,5), which is 0, not 0.5. So we need to find a way
#to change 0.5-->0. We take the second parameter (b) and 
#multiply it by 10, giving us 5. Now we subtract 5,
#so that a guess of b=0.5 transforms to b=0, as desired. 
#Test some other values of a,b,c to make sure you understand this step. 
#Would this same equation work for different bounds?...Nope.
def prior_transform(parameters):
    return np.array([5., 10., 10.]) * parameters + np.array([0., -5., 0.])

# Run nested sampling.
result_nest = nestle.sample(loglike, prior_transform, ndim=3,npoints=1000)

In [ ]:
#Let's see how we did
a_nest, b_nest,c_nest = np.percentile(result_nest.samples, 50,axis=0)
print('Chi:','a=',a_ml,'b=', b_ml,'c=',c_ml)
print('MCMC:','a=',a_mcmc,'b=',b_mcmc,'c=',c_mcmc)
print('Nested:','a=',a_nest,'b=',b_nest,'c=',c_nest)
print('True:','a=',a_true,'b=', b_true,'c=',c_true)
fig = corner.corner(result_nest.samples, 
                    labels=["$a$", "$b$","$c$"],
                    quantiles=(0.16,0.50, 0.84),
                    weights=result_nest.weights,#nested sampling provides weights for each sample based on bayesian evidence
                    truths=[a_true, b_true,c_true])
plt.show()

In [ ]:
#Let's see how Nested Sampling does for the "bad" case
def loglike(theta):
    args=(my_bad_data_1D['x'],my_bad_data_1D['y'],my_bad_data_1D['error'],my_bad_model)
    chisq = chisq_likelihood(theta,args)
    return -.5*chisq


def prior_transform(x):
    return np.array([5., 10., 10.]) * x + np.array([0., -5., 0.])

# Run nested sampling.
result_nest_bad = nestle.sample(loglike, prior_transform, 3,npoints=1000,maxcall=10000) #I know it'll have a difficult time converging, so I'm setting the maxcall to a finite number

a_nest_bad, b_nest_bad,c_nest_bad = np.percentile(result_nest_bad.samples, 50,axis=0)


In [ ]:
#Let's see how we did
print('Nested:','a=',a_nest_bad,'b=',b_nest_bad,'c=',c_nest_bad)
print('True:','a=',a_true_bad,'b=', b_true_bad,'c=',c_true_bad)
fig = corner.corner(result_nest_bad.samples, 
                    labels=["$a$", "$b$","$c$"],
                    weights=result_nest_bad.weights,
                    truths=[a_true_bad, b_true_bad,c_true_bad])
plt.show()
#The nested sampling perhaps did slightly better at mapping
#The full parameter space, but we could always add more 
#walkers (or samples per walker) for the MCMC

# Likelihood with Assumed PDF 

In [ ]:
#Let's consider one other problem. Here instead of defining
#A likelihood function and maximizing it, we want to 
#See how a theorist's model of a distribution does when
#we compare it to experiment.
#
#Suppose we have a distribution of energies that we are
#measuring. It has the following properties
true_energy_mean = 10
true_energy_std = 1
N_energies = 100

In [ ]:
np.random.seed(67)#Set the seed so everyone always gets the same data

#This will pull N_energies (100) points from a normal distribution
#with mean (loc) true_energy_mean, and standard deviation (scale)
#true_energy_std.
observed_energies=np.random.normal(loc=true_energy_mean,
                                   scale=true_energy_std,
                                   size=N_energies)
plt.hist(observed_energies)
plt.xlabel('Energy (GeV)')
plt.ylabel('Number')
plt.show()

In [ ]:
#Our theorist tells us that if we measure these energies,
#we're going to see a perfect Gaussian. We set that up here
def my_energy_model(energy_mean,energy_std,possible_energy):
        return(scipy.stats.norm.pdf(possible_energy,
                                    energy_mean,
                                    energy_std))

#Just see how this would actually compare before fitting
possible_energies=np.arange(0,20,.1)
model_energies=my_energy_model(10,1,possible_energies)
plt.hist(observed_energies,density=True)
plt.plot(possible_energies,model_energies)
plt.xlabel('Energy (GeV)')
plt.ylabel('Number')
plt.show()
#Not terrible.

## What is the all data likelihood function for all the data?

In [ ]:
def all_data_likelihood_function(model_parameters):
    model_mean,model_std = model_parameters
    #Since our model IS a PDF, a single point on it
    #would already be a likelihood. Therfore we can
    #just take the value of the PDF at all of the 
    #observed energies, with the fitter trying a
    #range of means and deviations, until we find
    #the most likely distribution. This is the same
    #as the commented code below, if it makes more
    #sense to you one point at a time.
    likelihoods = my_energy_model(model_mean,
                                  model_std,
                                  observed_energies)
    #likelihoods = 1.
    #for data_point in observed_energies:
    #    single_datapoint_likelihood = my_energy_model(model_mean,
    #                                                  model_std,
    #                                                  data_point)
    #    likelihoods *= single_datapoint_likelihood
    return(np.sum(np.log(likelihoods)))#we take the sum of the log instead of the product of the original likelihoods

In [ ]:
#Does it work for a single point? Does the likelihood go down
#If we move away from the true answer as it should? Yes.
all_data_likelihood = all_data_likelihood_function( ( 10, 1) )
print(all_data_likelihood)

In [ ]:
#See above for an explanation of the prior transform. This
#is the same as uniform bounds on our parameters of
#mean=(0,20), std=(0,5)
def prior_transform(x): 
    return np.array([20., 5.]) * x + np.array([0., 0.]) 

# Run nested sampling.
energy_result_nest = nestle.sample(all_data_likelihood_function, 
                            prior_transform, ndim=2,npoints=1000)

In [ ]:
#How was the theorist's prediction of the distribution? We
#Could also compare this to other distributions. For sanity
#We see that we recover the actual mean and standard deviations.
mean_nest, std_nest = np.percentile(energy_result_nest.samples, 50,axis=0)
print('Nested:','a=',mean_nest,'b=',std_nest)
print('True:','a=',true_energy_mean,'b=', true_energy_std)
fig = corner.corner(energy_result_nest.samples, 
                    labels=["$\mu$", "$\sigma$"],
                    weights=energy_result_nest.weights,
                    truths=[true_energy_mean, true_energy_std])
plt.show()